In [1]:
import pandas as pd
import numpy as np
import sys
import os
from netCDF4 import Dataset, date2num
import glob
import scipy.io
import tqdm
import xarray as xr
import os

In [2]:
from funciones_RFSM import *

# Generación de ficheros NETCF para las dinámicas

__En primer lugar vamos a generar los neccdf de las dinámicas con todos los puntos para de esta forma poder acceder a los datos más rápido.
Para ello configuramos las siguientes variables poniendo los modelos que se van a anlizar junto con los escenarios. Esto sólo es necesario ejecutarlo una
sola vez__

In [3]:
scenarios = ['his','rcp45','rcp85'] # Ponemos todos los escenarios que se van a analizar
models = ['DOW2'] # Ponemos todos los modelos de cambio climático además del ROW
path_output = 'P:/99_GRAFCAN CAMBIO CLIMATICO/01_BASES_DE_DATOS/SeriesReconstruidasPtosObjtvo/Netcdf/' # Path donde se van a guardar los ficheros netcdf
path_inputs = 'P:/99_GRAFCAN CAMBIO CLIMATICO/01_BASES_DE_DATOS/SeriesReconstruidasPtosObjtvo/' # Path donde se encuentran las dinámicas
path_install_RFSM = 'C:/IHRFSM_v1.0/' # Path donde se encuentra la instalación de RFSM
epsg = 32629 # Código EPSG del sistema de coordenadas en el que se está trabajando

__Generamos los ficheros de marea astronómica__

__Generamos los ficheros de Sea Level Rise__

__Generamos los ficheros de Oleaje__

In [4]:
for sc in scenarios:
    for m in models:
        if os.path.exists(path_output+'PtosObjtvo_DIN'+m+'_'+sc+'.nc'):
            continue
        else:
            try:
                create_netcf_Dinamicas_DOW(path_inputs,path_output,m,sc)
            except:
                print('Error en el modelo '+m+' en el escenario '+sc)
                continue

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Error en el modelo DOW2 en el escenario rcp45
Error en el modelo DOW2 en el escenario rcp85


# Generación de los ficheros de Cota de inundación para distitnos periodos de retorno

__A continuación se van a generar los ficheros para cada uno de los modelos y escenarios RCP 4.5 y RCP 8.5 en cada período de retorno que serán utilizados para generar las condiciones de contorno del modelo__

In [4]:
path_inputs_dinamicas = path_output
path_project          = 'F:/GRAFCAN/EH/RFSM/'
table_point_carac     = pd.read_csv(path_project+'ascii/Table_points_DIN.csv',index_col=0)
refNMMA               = 0
fin_year_period       = [2040,2070,2099]

In [ ]:
for i in models:
    generate_files_extrems_CC(path_inputs_dinamicas,i,path_project,table_point_carac,refNMMA,fin_year_period,cod_epsg=epsg)
    

__Generamos también los ficheros para el escenario ROW__

In [ ]:
generate_files_extrems(path_inputs_dinamicas,path_project,table_point_carac,refNMMA,cod_epsg=epsg)

# Generación de los casos a simular

__A continuación vamos a generar los diferentes casos a simular para la zona de estudio, teniendo en cuenta modelos, escenarios y períodos de retorno__

In [5]:
temporal          = [1.6, 2, 3,4, 3, 2, 1.6]
Table_impact_zone = pd.read_csv(path_project+'ascii/tblImpactZone.csv',index_col=0)
izcoast           = pd.read_csv(path_project+'ascii/izcoast.csv',index_col=0)
TestID            = 1
BCSetID           = 1
BCTypeID_COAST    = 2
return_period = [20,100,500]

In [6]:
StartTime = '0#h'
EndTime = str(len(temporal)+9)+'#h'
TimeStep = 1
SaveTimeStep = '20#m'
MaxTimeStep = 20
MinTimeStep = 0.0001
AlphaParameter = 0.7
ManningGlobalValue = 0.035
Results = '1'

### Caso escenario ROW

In [7]:
for T in return_period:
    file_TWL = path_project+'sites/'+'TWL_extrem_ROW.csv'
    #############  Ejecutamos el escenario ROW  ##############
    ##########################################################

    TestDesc = 'Sim_T_'+str(T)+'_'+'ROW'
    
    
    ## Creamos directorio
    make_dirs(path_install_RFSM,path_project,TestDesc,Results)
    
    ## Copiamos el fichero de rugosidad
    shutil.copy2(path_project+'ascii/tusrIZManning.csv', path_project+'tests/'+TestDesc+'/Input_User/')
    
    ### Generamos el fichero de condiciones de contorno
    tusrBCFlowLevel(table_point_carac,file_TWL,temporal,path_project,TestDesc,Table_impact_zone,
         izcoast,BCSetID,BCTypeID_COAST,T, SLR = None,
        raw_q=False, point_X_river = None ,point_Y_river = None ,hidrograma=None)

    ## Create XML
    create_xml(TestID,TestDesc,BCSetID,StartTime,EndTime,TimeStep,SaveTimeStep,MaxTimeStep,MinTimeStep,
               AlphaParameter,ManningGlobalValue,path_project,Results)
    ## Ejecutamos RFSM
    execute_RFSM(path_project,TestDesc,TestID)
    ## Generamos manchas de inundación
    export_result_RFSM_2(path_project,TestDesc,Results)

100%|███████████████████████████████████████████████████████████████████████████████| 588/588 [00:03<00:00, 172.00it/s]


Assuming windows!


100%|███████████████████████████████████████████████████████████████████████████████| 588/588 [00:03<00:00, 177.06it/s]


Assuming windows!


100%|███████████████████████████████████████████████████████████████████████████████| 588/588 [00:03<00:00, 151.08it/s]


Assuming windows!


100%|██████████████████████████████████████████████████████████████████████████████| 742/742 [00:00<00:00, 1585.48it/s]


### Casos de cambio climático

In [ ]:
for m in models:
    for sc in scenarios:
        for P in fin_year_period:
            for T in return_period:
                if sc =='his':
                    file_TWL = path_project+'sites/CC/'+'TWL_extrem_'+model+'_his.csv'
                    #############  Ejecutamos el período histórico  ##############
                    ##############################################################

                    TestDesc = 'Sim_T_'+str(T)+'_'+m+'_his'
                    
                     ## Creamos directorio
                    make_dirs(path_install_RFSM,path_project,TestDesc)
                    
                     ## Copiamos el fichero de rugosidad
                    shutil.copy2(path_project+'ascii/tusrIZManning.csv', path_project+'tests/'+TestDesc+'/Input_User/')
                    
                    ### Generamos el fichero de condiciones de contorno
                    tusrBCFlowLevel(table_point_carac,file_TWL,temporal,path_project,TestDesc,Table_impact_zone,
                         izcoast,BCSetID,BCTypeID_COAST,T, SLR = None,
                        raw_q=False, point_X_river = None ,point_Y_river = None ,hidrograma=None)
                    ## Create XML
                    create_xml(TestID,TestDesc,BCSetID,StartTime,EndTime,TimeStep,SaveTimeStep,MaxTimeStep,MinTimeStep,
                               AlphaParameter,ManningGlobalValue,path_project,Results)
                    ## Ejecutamos RFSM
                    execute_RFSM(path_project,TestDesc,TestID)
                    ## Generamos manchas de inundación
                    export_result_RFSM_2(path_project,TestDesc,Results)

                else:
                    file_TWL = path_project+'sites/CC/'+'TWL_extrem_'+model+'_RCP45_'+str(P)+'_.csv'
                    ### Ejecutamos los diferentes escenarios de SLR

                    #########################  MSLR  #############################
                    ##############################################################
                    TestDesc = 'Sim_T_'+str(T)+'_'+m+'_'+sc+'_'+'MSLR'
                    
                    ## Creamos directorio
                    make_dirs(path_install_RFSM,path_project,TestDesc)
                    
                     ## Copiamos el fichero de rugosidad
                    shutil.copy2(path_project+'ascii/tusrIZManning.csv', path_project+'tests/'+TestDesc+'/Input_User/')
                    
                    ### Generamos el fichero de condiciones de contorno
                    tusrBCFlowLevel(table_point_carac,file_TWL,temporal,path_project,TestDesc,Table_impact_zone,
                         izcoast,BCSetID,BCTypeID_COAST,T,SLR='MSLR',
                        raw_q=False, point_X_river = None ,point_Y_river = None ,hidrograma=None)
                    ## Ejecutamos RFSM
                    execute_RFSM(path_project,TestDesc,TestID)
                    ## Generamos manchas de inundación
                    export_result_RFSM_2(path_project,TestDesc,Results)

                    #########################  SLR_5  #############################
                    ###############################################################
                    TestDesc = 'Sim_T_'+str(T)+'_'+m+'_'+sc+'_'+'SLR_5'
                    ## Creamos directorio
                    make_dirs(path_install_RFSM,path_project,TestDesc)
                    
                    ## Copiamos el fichero de rugosidad
                    shutil.copy2(path_project+'ascii/tusrIZManning.csv', path_project+'tests/'+TestDesc+'/Input_User/')
                    
                    ### Generamos el fichero de condiciones de contorno
                    tusrBCFlowLevel(table_point_carac,file_TWL,temporal,path_project,TestDesc,Table_impact_zone,
                         izcoast,BCSetID,BCTypeID_COAST,T,SLR='SLR_5',
                        raw_q=False, point_X_river = None ,point_Y_river = None ,hidrograma=None)
                    ## Ejecutamos RFSM
                    execute_RFSM(path_project,TestDesc,TestID)
                    ## Generamos manchas de inundación
                    export_result_RFSM_2(path_project,TestDesc,Results)

                    #########################  SLR_95  #############################
                    ################################################################

                    TestDesc = 'Sim_T_'+str(T)+'_'+m+'_'+sc+'_'+'SLR_95'
                    ## Creamos directorio
                    make_dirs(path_install_RFSM,path_project,TestDesc)
                    
                     ## Copiamos el fichero de rugosidad
                    shutil.copy2(path_project+'ascii/tusrIZManning.csv', path_project+'tests/'+TestDesc+'/Input_User/')
                    
                    ### Generamos el fichero de condiciones de contorno
                    tusrBCFlowLevel(table_point_carac,file_TWL,temporal,path_project,TestDesc,Table_impact_zone,
                         izcoast,BCSetID,BCTypeID_COAST,T,SLR='SLR_95',
                        raw_q=False, point_X_river = None ,point_Y_river = None ,hidrograma=None)
                    ## Ejecutamos RFSM
                    execute_RFSM(path_project,TestDesc,TestID)
                    ## Generamos manchas de inundación
                    export_result_RFSM_2(path_project,TestDesc,Results)